# Aggregation in Pandas
One of the many benefits of using Pandas is how easily Pandas makes it to group and aggregate data.  
Data aggregation is a common task that you may have to perform as part of data analysis and this can be accomplished quite easily using Pandas; inbuilt functions.

### Pandas Describe Function
This function gives a general descriptive statistics for a DataFrame/Series for all numerical columns.
Let's use the [Pima Indians Dataset](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database) and it can be loaded from [here](https://raw.githubusercontent.com/npradaschnor/Pima-Indians-Diabetes-Dataset/master/diabetes.csv). You can use the URL to load the data directly

In [ ]:
# Import libraries
import pandas as pd

In [ ]:
# Load the data
filename="https://raw.githubusercontent.com/npradaschnor/Pima-Indians-Diabetes-Dataset/master/diabetes.csv"
df = pd.read_csv(filename)

In [ ]:
# Basic exploration
df.head(3)

In [ ]:
# What else can we check?
df.info()

In [ ]:
# Descriptive statistics with describe() method
df.describe()

Note that this summary will be done for all numerical columns even if the underlying data is not numerical in type so you may need to slice some columns.

In [ ]:
# Describe while specifying precision
df.describe().round(3)

You can also get individual measures from each column.  
Let us consider the BloodPressure column

In [ ]:
# Slice the clomun
bp_data = df['BloodPressure']

In [ ]:
# Get the mean
bp_data.mean()

In [ ]:
# Get the median
bp_data.median()

In [ ]:
# Get the variance
bp_data.var()

In [ ]:
# Get the standard deviation
bp_data.std()

In [ ]:
# Minimum
bp_data.min()

In [ ]:
# Maximum
bp_data.max()

In [14]:
# What is the range

# Calculate the range of the 'BloodPressure' column
bp_range = bp_data.max() - bp_data.min()

# Print the result
print("Range of BloodPressure:", bp_range)

Range of BloodPressure: 122


In [15]:
# Getting the mode
bp_data.mode()

0    70
Name: BloodPressure, dtype: int64

Let's use this  [data](https://drive.google.com/file/d/1wmbwbnZww7cue1FYr7aeATRAD611Kgla/view?usp=sharing) and with Pandas to answer several questions.

In [ ]:
# Connecting to google drive
# from google.colab import drive
# drive.mount('/content/drive')

In [16]:
# Import Pandas
import pandas as pd

In [17]:
# Load the data
filename1 = '../files/Sales.csv'
df = pd.read_csv(filename1)

df.head()

,Item,Student,Packaging,Price,Expense,Quantity Sold (Day 1),Quantity Sold (Day 2),Quantity Sold (Day 3),Quantity Sold (Day 4)
0,brownie,Hugo,Individually Wrapped (Plastic),2.25,0.25,17,19,25,25
1,cookie,Sally,Individually Wrapped (Plastic),1.25,0.50,40,32,38,38
2,cake,Martina,Boxed (Clear Plastic),9.50,5.00,1,2,0,0
3,cupcake,Joe,Boxed (Cardboard),3.50,0.75,10,14,12,12
4,fudge,Hugo,Individually Wrapped (Foil),3.00,1.00,0,20,22,22


# Stakeholder Questions

The club wants to reward students based on their contributions to the bake sale. The club needs answers to the following questions in order to determine the appropriate awards:

1) How many different types of items did each student create for the bake sale? Which student provided the most Items?

2) What was the total quantity of baked goods sold, by item? Which item sold the largest quantity?

3) What were the total profits generated by each Item? Which Item generated the most total profit?

4) Which student sold the largest quantity of items?

5) Which student generated the most total profit?

6) How much money has the club earned overall by the end of Day 4? How much more do they need to earn on Day 5 to meet their goal?

In [18]:
# Let us first set the item as the new index
df.set_index('Item', inplace=True)

In [19]:
df.head()

,Student,Packaging,Price,Expense,Quantity Sold (Day 1),Quantity Sold (Day 2),Quantity Sold (Day 3),Quantity Sold (Day 4)
Item,,,,,,,,
brownie,Hugo,Individually Wrapped (Plastic),2.25,0.25,17,19,25,25
cookie,Sally,Individually Wrapped (Plastic),1.25,0.50,40,32,38,38
cake,Martina,Boxed (Clear Plastic),9.50,5.00,1,2,0,0
cupcake,Joe,Boxed (Cardboard),3.50,0.75,10,14,12,12
fudge,Hugo,Individually Wrapped (Foil),3.00,1.00,0,20,22,22


**1) How many different types of items did each student create for the bake sale? Which student provided the most Items?**  
We can use the **.value_counts()** method which returns the unique occurances of each value

In [20]:
# Get the unique values and their count for the Student column
df["Student"].value_counts()

Student
Martina    3
Hugo       2
Sally      2
Jade       2
Joe        1
Mark       1
Anne       1
Name: count, dtype: int64

In [21]:
# Use normalize = True to output a ratio/ fraction
df["Student"].value_counts(normalize=True)


Student
Martina    0.250000
Hugo       0.166667
Sally      0.166667
Jade       0.166667
Joe        0.083333
Mark       0.083333
Anne       0.083333
Name: proportion, dtype: float64

In [22]:
# proof
df["Student"].value_counts(normalize=True).sum()

1.0

2) What was the total quantity of baked goods sold, by item? Which item sold the largest quantity?  

In order for us to answer the next question about which student sold the largest quantity of items across days 1-4.

We can calculate this by leveraging Pandas' integration of NumPy aggregation functions/methods.

Pandas has many aggregation methods included, here is a table with the most common aggregation functions.
<img src="https://assets.codingdojo.com/boomyeah2015/codingdojo/curriculum/content/chapter/1680737011__Capture.PNG">

We want to calculate the total sales for the 4 days of the bake sale thus far and to calculate the club's total profit the evening before the final day of the bake sale.

Let's first apply slicing and filtering to create a list of column names that only contain the word "Quantity".

In [23]:
# select quantity columns using .str.contains
columns = df.columns
filter_quantity_cols = columns[columns.str.contains('Quantity')]
filter_quantity_cols

Index(['Quantity Sold (Day 1)', 'Quantity Sold (Day 2)',
       'Quantity Sold (Day 3)', 'Quantity Sold (Day 4)'],
      dtype='object')

In [24]:
# Filter the dataframe
df[filter_quantity_cols]

,Quantity Sold (Day 1),Quantity Sold (Day 2),Quantity Sold (Day 3),Quantity Sold (Day 4)
Item,,,,
brownie,17,19,25,25
cookie,40,32,38,38
cake,1,2,0,0
cupcake,10,14,12,12
fudge,0,20,22,22
banana bread,0,10,13,13
torte,0,5,7,7
scone,0,8,10,10
muffin,0,3,4,4


In [25]:
# Get the sum by column
df[filter_quantity_cols].sum()

Quantity Sold (Day 1)     68
Quantity Sold (Day 2)    113
Quantity Sold (Day 3)    146
Quantity Sold (Day 4)    167
dtype: int64

In [26]:
# Obtain the sum across rows by passing 1 for the axis argument
df[filter_quantity_cols].sum(axis=1)

Item
brownie                  86
cookie                  148
cake                      3
cupcake                  48
fudge                    64
banana bread             36
torte                    19
scone                    28
muffin                   11
rice krispies treats     40
apple pie                 5
key lime pie              6
dtype: int64

In [27]:
# Add a column to include the cumulative quantity
df['Cumulative Quantity (Day 4)'] = df[filter_quantity_cols].sum(axis=1)
df.head()

,Student,Packaging,Price,Expense,Quantity Sold (Day 1),Quantity Sold (Day 2),Quantity Sold (Day 3),Quantity Sold (Day 4),Cumulative Quantity (Day 4)
Item,,,,,,,,,
brownie,Hugo,Individually Wrapped (Plastic),2.25,0.25,17,19,25,25,86
cookie,Sally,Individually Wrapped (Plastic),1.25,0.50,40,32,38,38,148
cake,Martina,Boxed (Clear Plastic),9.50,5.00,1,2,0,0,3
cupcake,Joe,Boxed (Cardboard),3.50,0.75,10,14,12,12,48
fudge,Hugo,Individually Wrapped (Foil),3.00,1.00,0,20,22,22,64


In [28]:
# Sort the cumulative values from high to low for better presentation
df['Cumulative Quantity (Day 4)'].sort_values(ascending=False)

Item
cookie                  148
brownie                  86
fudge                    64
cupcake                  48
rice krispies treats     40
banana bread             36
scone                    28
torte                    19
muffin                   11
key lime pie              6
apple pie                 5
cake                      3
Name: Cumulative Quantity (Day 4), dtype: int64

**3) What were the total profits generated by each Item? Which Item generated the most total profit?**  
We shall first get the profit for each item.  

To calculate the cumulative profit for each baked good at the end of day 4,  multiply the cumulative quantity column by our Profit Per Item Column.   
We will add the result to a new column in the dataframe called "Cumulative Profit (Day 4)"

In [29]:
# Getting Profit per Item
df["Profit per Item"] = df['Price'] - df['Expense']

In [30]:
df.head(3)

,Student,Packaging,Price,Expense,Quantity Sold (Day 1),Quantity Sold (Day 2),Quantity Sold (Day 3),Quantity Sold (Day 4),Cumulative Quantity (Day 4),Profit per Item
Item,,,,,,,,,,
brownie,Hugo,Individually Wrapped (Plastic),2.25,0.25,17,19,25,25,86,2.00
cookie,Sally,Individually Wrapped (Plastic),1.25,0.50,40,32,38,38,148,0.75
cake,Martina,Boxed (Clear Plastic),9.50,5.00,1,2,0,0,3,4.50


In [31]:
# Multiply two cumulative quantity by profit per item to get cumulative profit
df['Cumulative Profit (Day 4)'] = df['Cumulative Quantity (Day 4)']  * df['Profit per Item']
df.head()

,Student,Packaging,Price,Expense,Quantity Sold (Day 1),Quantity Sold (Day 2),Quantity Sold (Day 3),Quantity Sold (Day 4),Cumulative Quantity (Day 4),Profit per Item,Cumulative Profit (Day 4)
Item,,,,,,,,,,,
brownie,Hugo,Individually Wrapped (Plastic),2.25,0.25,17,19,25,25,86,2.00,172.0
cookie,Sally,Individually Wrapped (Plastic),1.25,0.50,40,32,38,38,148,0.75,111.0
cake,Martina,Boxed (Clear Plastic),9.50,5.00,1,2,0,0,3,4.50,13.5
cupcake,Joe,Boxed (Cardboard),3.50,0.75,10,14,12,12,48,2.75,132.0
fudge,Hugo,Individually Wrapped (Foil),3.00,1.00,0,20,22,22,64,2.00,128.0


In [32]:
# Show the cumulative profit in ascending order?
df['Cumulative Profit (Day 4)'].sort_values(ascending=False)

Item
brownie                 172.00
cupcake                 132.00
fudge                   128.00
cookie                  111.00
torte                    95.00
banana bread             81.00
rice krispies treats     40.00
apple pie                30.00
key lime pie             30.00
scone                    21.00
cake                     13.50
muffin                    8.25
Name: Cumulative Profit (Day 4), dtype: float64

**4) Which student sold the largest quantity of items?**  
To address this question we will group our data by Student. We use .sum() as the aggregation calculation.

**Pandas Groupby**  
Pandas makes it very easy to calculate aggregations grouped by one or more columns.

To use groupby:

* start with a dataframe (not a Series/single column)
* followed by groupby()with the names of the column(s) included in the parenthesis
* an aggregation method such as .sum()

In [33]:
# groupby needs an aggregation functon
student_sums = df.groupby("Student").sum()
student_sums

,Packaging,Price,Expense,Quantity Sold (Day 1),Quantity Sold (Day 2),Quantity Sold (Day 3),Quantity Sold (Day 4),Cumulative Quantity (Day 4),Profit per Item,Cumulative Profit (Day 4)
Student,,,,,,,,,,
Anne,Individually Wrapped (Foil),1.50,0.75,0,3,4,4,11,0.75,8.25
Hugo,Individually Wrapped (Plastic)Individually Wr...,5.25,1.25,17,39,47,47,150,4.00,300.00
Jade,Boxed (Clear Plastic)Individually Wrapped (Pl...,12.50,6.75,0,13,17,17,47,5.75,116.00
Joe,Boxed (Cardboard),3.50,0.75,10,14,12,12,48,2.75,132.00
Mark,Individually Wrapped (Foil),2.75,0.50,0,10,13,13,36,2.25,81.00
Martina,Boxed (Clear Plastic)Individually Wrapped (Pl...,20.25,9.75,1,2,13,33,49,10.50,83.50
Sally,Individually Wrapped (Plastic)Box (Cardboard),11.25,4.50,40,32,40,41,153,6.75,141.00


In [34]:
# Obtain the only the cumulative profit per student by slicing
quantity_by_student = df.groupby('Student')['Cumulative Quantity (Day 4)'].sum()
quantity_by_student

Student
Anne        11
Hugo       150
Jade        47
Joe         48
Mark        36
Martina     49
Sally      153
Name: Cumulative Quantity (Day 4), dtype: int64

In [35]:
# Sort from highest to lowest profit
quantity_by_student.sort_values(ascending=False)

Student
Sally      153
Hugo       150
Martina     49
Joe         48
Jade        47
Mark        36
Anne        11
Name: Cumulative Quantity (Day 4), dtype: int64

**5) Which student generated the most total profit?**

In [36]:
# Obtain cumulative profit grouped by student
profit_by_student = df.groupby('Student')['Cumulative Profit (Day 4)'].sum()
# Sort from highest to lowest
profit_by_student.sort_values(ascending=False)

Student
Hugo       300.00
Sally      141.00
Joe        132.00
Jade       116.00
Martina     83.50
Mark        81.00
Anne         8.25
Name: Cumulative Profit (Day 4), dtype: float64

**6) How much money has the club earned overall by the end of Day 4? How much more do they need to earn on Day 5 to meet their goal?**


In [37]:
# Total Profit for the club
total_profit = df['Cumulative Profit (Day 4)'].sum()
print(total_profit)

861.75


In [40]:
# How much more money do they need to earn 1000?
# Total Profit for the club
total_earnings_day_4 = df['Cumulative Profit (Day 4)'].sum()
club_goal = 1000  # Assuming the club's goal is 1000
remaining_goal_day_5 = club_goal - total_earnings_day_4
print(f"Club earnings overall by the end of Day 4: {total_earnings_day_4}\nRemaining earnings needed on Day 5 to meet the goal: {remaining_goal_day_5}\n")

Club earnings overall by the end of Day 4: 861.75
Remaining earnings needed on Day 5 to meet the goal: 138.25

